In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import string
%matplotlib inline

In [2]:
# Setting
filename = "All_New_1"
filename_KFold = '1'
MAX_LEN = 500 #due to the max length of the token [most BERT agree 512 token] => 512 without features or labels in it
# batch size = 16, 32
batch_size = 4
WangChanBERTa_Model = True
premodel_list = ['xlm-roberta-base','xlm-roberta-large']
XLMR_chosen_premodel = premodel_list[1]
# Model_path
WangChanBERTa_path = "../WangChanBERT/Model/WangChanBERTa_All_New_1_epoch4.h5"
XLMR_path = "../XLMR/Model/XLMR_All_New_1_epoch4.h5"

In [3]:
if torch.cuda.is_available(): 
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3090


In [4]:
df_crime = pd.read_csv('../Data/K_Fold_Dataset/1_Train.csv')
df_crime_test = pd.read_csv('../Data/K_Fold_Dataset/1_Test.csv')

In [5]:
X_train = df_crime.iloc[:,3:4].values
Y_train = df_crime.iloc[:,5:].values
X_test = df_crime_test.iloc[:,3:4].values
Y_test = df_crime_test.iloc[:,5:].values

In [6]:
from skmultilearn.model_selection import iterative_train_test_split
X_train, Y_train, X_val, Y_val = iterative_train_test_split(X_train,Y_train,test_size=0.09)

In [8]:
# !pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForMaskedLM, CamembertTokenizer, XLMRobertaTokenizer
# cstorm125/wangchanberta-base-att-spm-uncased-finetune
# airesearch/wangchanberta-base-att-spm-uncased
tokenizer = XLMRobertaTokenizer.from_pretrained(XLMR_chosen_premodel)

def preprocessing_for_bert(data):

    input_ids = []
    attention_masks = []

    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            #text=text_preprocessing(sent),  
            text=sent,
            add_special_tokens=True,        
            max_length=MAX_LEN,
            truncation=True,             
            padding='max_length',         
            #return_tensors='pt',           
            return_attention_mask=True      
        )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [9]:
X_val_new = np.array([text for sub in X_val for text in sub]) 
val_inputs, val_masks = preprocessing_for_bert(X_val_new)

In [10]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

val_labels = torch.FloatTensor(Y_val)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
X_test_new = np.array([text for sub in X_test for text in sub]) 
test_inputs, test_masks = preprocessing_for_bert(X_test_new)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

In [11]:
import torch.nn.functional as F
from transformers import CamembertForSequenceClassification , BertForSequenceClassification, XLMRobertaForSequenceClassification
import torch.nn as nn
import torch
bert_classifier_load = XLMRobertaForSequenceClassification.from_pretrained(XLMR_chosen_premodel,num_labels=8,output_attentions=False,output_hidden_states=False)
bert_classifier_load = nn.DataParallel(bert_classifier_load)
bert_classifier_load.load_state_dict(torch.load(XLMR_path))
bert_classifier_load.to(device)
bert_classifier_load.eval()

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            outputs = model(b_input_ids, b_attn_mask)
            logits = outputs.logits
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = all_logits.sigmoid().cpu().numpy()

    return probs

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [12]:
probs = bert_predict(bert_classifier_load, test_dataloader)

In [13]:
probs

array([[1.8006079e-04, 5.8390307e-03, 1.0350996e-03, ..., 4.2395008e-01,
        5.0483592e-04, 2.0129347e-02],
       [9.3506451e-04, 9.9796104e-01, 5.9458392e-04, ..., 3.5353355e-02,
        1.9531608e-02, 1.7653332e-03],
       [3.8218003e-04, 9.8062551e-01, 4.1349772e-03, ..., 2.5090918e-01,
        1.7940446e-03, 1.8690749e-03],
       ...,
       [2.2698715e-03, 9.9512804e-01, 5.5760005e-03, ..., 3.0303210e-01,
        2.9117796e-03, 1.7520966e-03],
       [6.9855514e-04, 6.3155582e-03, 1.7087327e-03, ..., 9.8451567e-01,
        1.1105060e-03, 4.6853572e-03],
       [1.5995199e-03, 2.2404182e-03, 2.5139085e-03, ..., 3.7545380e-03,
        2.2902121e-03, 2.5515447e-03]], dtype=float32)

In [14]:
threshold_set = [0.9,0.5,0.5,0.2,0.7,0.5,0.6,0.3]

In [15]:
probs[...,0] = np.where(probs[...,0] > threshold_set[0],1,0)
probs[...,1] = np.where(probs[...,1] > threshold_set[1],1,0)
probs[...,2] = np.where(probs[...,2] > threshold_set[2],1,0)
probs[...,3] = np.where(probs[...,3] > threshold_set[3],1,0)
probs[...,4] = np.where(probs[...,4] > threshold_set[4],1,0)
probs[...,5] = np.where(probs[...,5] > threshold_set[5],1,0)
probs[...,6] = np.where(probs[...,6] > threshold_set[6],1,0)
probs[...,7] = np.where(probs[...,7] > threshold_set[7],1,0)

In [16]:
from sklearn.metrics import multilabel_confusion_matrix

In [17]:
multilabel_confusion_matrix(Y_test,probs)

array([[[829,   3],
        [  1,  24]],

       [[577,  24],
        [ 23, 233]],

       [[782,   7],
        [  4,  64]],

       [[766,  14],
        [ 11,  66]],

       [[742,  11],
        [ 12,  92]],

       [[614,  54],
        [ 50, 139]],

       [[770,  15],
        [  9,  63]],

       [[702,  14],
        [ 20, 121]]])

In [18]:
Y_test

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [81]:
news_index = []
ground_truth_set = [[],[],[],[],[],[],[],[]]
prediction_set = [[],[],[],[],[],[],[],[]]
for index in range(len(Y_test)):
    if (Y_test[index][5] != probs[index][5]):
        news_index.append(index)
        for i in range(8):
            ground_truth_set[i].append(Y_test[index][i])
            prediction_set[i].append(int(probs[index][i]))
        print(f'{index} {Y_test[index]} {probs[index]}')

8 [0 0 1 0 0 0 0 0] [0. 1. 1. 0. 0. 1. 0. 0.]
12 [0 1 0 0 0 0 0 0] [0. 1. 0. 0. 0. 1. 0. 0.]
17 [0 1 1 0 0 0 0 0] [0. 1. 1. 0. 0. 1. 0. 0.]
24 [0 0 1 0 0 0 0 0] [0. 1. 1. 0. 0. 1. 0. 0.]
26 [0 0 0 0 0 0 1 0] [0. 1. 0. 0. 0. 1. 0. 0.]
31 [0 1 0 0 0 0 0 0] [0. 0. 0. 0. 0. 1. 0. 0.]
32 [0 1 0 0 0 1 0 0] [0. 1. 0. 0. 0. 0. 0. 0.]
39 [0 0 1 0 0 0 0 0] [0. 0. 1. 0. 0. 1. 0. 0.]
41 [0 0 0 1 0 0 0 0] [0. 0. 0. 1. 0. 1. 0. 0.]
52 [0 1 0 0 1 0 0 0] [0. 1. 0. 0. 1. 1. 0. 0.]
53 [0 1 0 0 0 1 0 0] [0. 1. 0. 0. 0. 0. 0. 0.]
54 [0 1 0 1 0 1 0 0] [0. 1. 0. 1. 0. 0. 0. 0.]
60 [0 1 0 0 0 0 0 0] [0. 1. 0. 0. 0. 1. 0. 0.]
63 [0 1 0 0 0 1 0 0] [0. 1. 0. 0. 0. 0. 0. 0.]
71 [0 1 0 0 0 1 0 0] [0. 1. 0. 0. 0. 0. 0. 0.]
76 [0 1 0 1 0 1 0 0] [0. 1. 0. 1. 0. 0. 0. 0.]
79 [0 0 0 0 0 1 0 0] [0. 0. 0. 0. 0. 0. 0. 1.]
105 [0 1 1 1 0 0 0 0] [0. 0. 0. 0. 0. 1. 0. 0.]
113 [0 0 0 0 0 0 1 0] [0. 0. 0. 0. 0. 1. 0. 0.]
122 [0 1 0 1 0 1 0 0] [0. 1. 0. 1. 0. 0. 0. 0.]
137 [0 1 0 0 1 0 0 0] [0. 1. 0. 0. 1. 1. 0. 0.]
142 [0 0 0

In [82]:
bat_assault_probs = []
for news in news_index:
    bat_assault_probs.append(int(probs[news][5]))

In [83]:
len(news_index)

104

In [84]:
df_crime_output = df_crime_test.iloc[news_index]

In [85]:
df_crime_output = df_crime_output.drop(['All_New_Format'],axis=1)
# df_crime_output = df_crime_output.rename(columns={'Battery/Assault':'Ground Truth'})

In [86]:
df_crime_output = df_crime_output.assign(Pred_Gambling=prediction_set[0],Pred_Murder=prediction_set[1],Pred_SexualAbuse=prediction_set[2],Pred_Theft_Burglary=prediction_set[3],Pred_Drug=prediction_set[4],Pred_Battery_Assault=prediction_set[5],Pred_Accident=prediction_set[6],Pred_Non_Crime=prediction_set[7])
# df_crime_output = df_crime_output.assign(Ground_Truth_set=ground_truth_set)
# df_crime_output = df_crime_output.assign(Prediction_set=prediction_set)

In [87]:
df_crime_output

,News Title,News_Intro,News_Desc,News_All,Gambling,Murder,Sexual Abuse,Theft/Burglary,Drug,Battery/Assault,Accident,Non-Crime,Pred_Gambling,Pred_Murder,Pred_SexualAbuse,Pred_Theft_Burglary,Pred_Drug,Pred_Battery_Assault,Pred_Accident,Pred_Non_Crime
8,ซิวแล้วหนุ่มหื่นจี้ข่มขืนสาว ชี้พ้นโทษ3ปีก่อเห...,โปลิศอุบลราชธานี ตามจับ &quot;หนุ่มหื่น&quot; ...,จากกรณีเหตุ คนร้ายบุกใช้มีดจี้ข่มขืน น.ส.เอ (น...,ซิวแล้วหนุ่มหื่นจี้ข่มขืนสาว ชี้พ้นโทษ3ปีก่อเห...,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0
12,ตร.หิ้ว&quot;คม แหลมสอม&quot; สอบคดียิงถล่มผับ...,เมียนมาส่งตัวคืน&quot;คม แหลมสอม&quot; ตร.รับต...,จากกรณี นายชยพล หรืออาคม เสียมไหม อายุ 35ปี ฉา...,ตร.หิ้ว&quot;คม แหลมสอม&quot; สอบคดียิงถล่มผับ...,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
17,หนุ่มปริศนาพาสาวโอเกะ นั่งจยย.หายไปก่อนตายเปลือย,"ตำรวจเผยข้อมูล ""สาวคาราโอเกะ"" ก่อนถูกข่มขืนตาย...",กรณีนางพิศ เคียนประโดม สาวพนักงานร้านคาราโอเกะ...,หนุ่มปริศนาพาสาวโอเกะ นั่งจยย.หายไปก่อนตายเปลื...,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0
24,อมนุษย์!ฆ่าทุบหัว'ยาย'ก่อนข่มขืน เจ้าตูบพา'ตา'...,โจรหื่นสุดเหี้ยม ลากยายวัย 72 เข้าไปในสวนปาล์ม...,เมื่อวันที่ 3 มี.ค. ร.ต.อ.วิฑูรย์ เผือกวารี พง...,อมนุษย์!ฆ่าทุบหัว'ยาย'ก่อนข่มขืน เจ้าตูบพา'ตา'...,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0
26,จับหลวงพี่เมาขับรถ พุ่งชนคนงานตาย 1 สาหัส 1 ตะ...,พระเมาซิ่งเก๋ง พุ่งชนคนงานก่อสร้างถนนตาย 1 สาห...,พระเมาซิ่งเก๋ง พุ่งชนคนงานก่อสร้างถนนตาย 1 สาห...,จับหลวงพี่เมาขับรถ พุ่งชนคนงานตาย 1 สาหัส 1 ตะ...,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,เตือนภัยฝรั่งบ้ากาม ขี่รถตระเวนก่อเหตุ ข่มขืนห...,ฝรั่งโรคจิต รายเดียวกับที่ทำร้ายร่างกาย พยายาม...,ฝรั่งโรคจิต รายเดียวกับที่ทำร้ายร่างกาย พยายาม...,เตือนภัยฝรั่งบ้ากาม ขี่รถตระเวนก่อเหตุ ข่มขืนห...,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
817,2ชายพกปืนป่วนงานแต่งซึม ศาลไม่ให้ประกันนอนคุก,นอนคุกไปตามระเบียบ ศาลไม่ให้ประกัน '2ชาย'พกปืน...,เมื่อวันที่ 12 มี.ค.เจ้าหน้าที่ตำรวจ สภ.สบปราบ...,2ชายพกปืนป่วนงานแต่งซึม ศาลไม่ให้ประกันนอนคุก ...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
825,ซุ่มยิงดับเฒ่า ขณะทวงหนี้ญาติ ลูกหนี้ตีมึนปัดไ...,เฒ่า 62 ปืนดุ ย่องควงลูกซอง ยิงปากเฒ่า 67 ดับค...,เฒ่า 62 ปืนดุ ย่องควงลูกซอง ยิงปากเฒ่า 67 ดับค...,ซุ่มยิงดับเฒ่า ขณะทวงหนี้ญาติ ลูกหนี้ตีมึนปัดไ...,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
832,ง้อขอคืนดีเมียสาวไม่สำเร็จ ค้อนทุบดับ!ผูกคอตายตาม,หนุ่มรับเหมาก่อสร้างเคลียร์ปัญหาความรักกันไม่ล...,เมื่อวันที่ 25 เม.ย. ร.ต.อ.เฉลิม ทองน้อย รองสว...,ง้อขอคืนดีเมียสาวไม่สำเร็จ ค้อนทุบดับ!ผูกคอตาย...,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [88]:
df_crime_output.to_excel('./Result.xlsx',encoding='utf-8-sig',index= False)